In [8]:
from typing import TypedDict, NotRequired, Optional, cast
from discord.types.embed import Embed as DiscordEmbed

class StreamData(TypedDict):
    id: str
    user_id: str
    user_login: str
    user_name: str
    game_id: str
    game_name: str
    type: str
    title: str
    viewer_count: int
    started_at: str
    language: str
    thumbnail_url: str
    tag_ids: list[str]
    tags: list[str]
    is_mature: bool

stream_data: StreamData = {
    "id": "314179810258",
    "user_id": "1315751183",
    "user_login": "escaperoom1_gymnham",
    "user_name": "escaperoom1_gymnham",
    "game_id": "509663",
    "game_name": "Special Events",
    "type": "live",
    "title": "Escaperoom Test",
    "viewer_count": 0,
    "started_at": "2025-08-17T22:38:49Z",
    "language": "de",
    "thumbnail_url": "https://static-cdn.jtvnw.net/previews-ttv/live_user_escaperoom1_gymnham-{width}x{height}.jpg",
    "tag_ids": [],
    "tags": ["Deutsch"],
    "is_mature": False,
}


class UserData(TypedDict):
    id: str
    login: str
    display_name: str
    type: str
    broadcaster_type: str
    description: str
    profile_image_url: str
    offline_image_url: str
    view_count: int
    email: NotRequired[str]
    created_at: str

user_data: UserData = {
    "id": "1315751183",
    "login": "escaperoom1_gymnham",
    "display_name": "escaperoom1_gymnham",
    "type": "",
    "broadcaster_type": "",
    "description": "",
    "profile_image_url": "https://static-cdn.jtvnw.net/user-default-pictures-uv/cdd517fe-def4-11e9-948e-784f43822e80-profile_image-300x300.png",
    "offline_image_url": "",
    "view_count": 0,
    "created_at": "2025-05-25T23:49:43Z",
}


class DiscordNotificationMessage(TypedDict, total=False):
    content: str
    embed: DiscordEmbed


def replace_variables_in_notification(
    message_data: DiscordNotificationMessage, stream_data: Optional[StreamData], user_data: Optional[UserData]
) -> DiscordNotificationMessage:
    """
    Recursively replace all variables in a DiscordNotificationMessage
    with values from stream_data and user_data.

    Format: $user.var_name for user data, $stream.var_name for stream data

    #### User Data Variables (from UserData):
    $user.id - User ID
    $user.login - Username (login name)
    $user.display_name - Display name
    $user.type - User type
    $user.broadcaster_type - Broadcaster type
    $user.description - User description
    $user.profile_image_url - Profile image URL
    $user.offline_image_url - Offline image URL
    $user.view_count - View count
    $user.email - Email
    $user.created_at - Account creation date
    #### Stream Data Variables (from StreamData):
    $stream.id - Stream ID
    $stream.user_id - User ID
    $stream.user_login - Username
    $stream.user_name - User display name
    $stream.game_id - Game ID
    $stream.game_name - Game name
    $stream.type - Stream type
    $stream.title - Stream title
    $stream.tags - Stream tags
    $stream.viewer_count - Current viewer count
    $stream.started_at - Stream start time
    $stream.language - Stream language
    $stream.thumbnail_url - Raw thumbnail URL
    $stream.is_mature - Mature content flag
    #### Special Variables:
    $stream_url - Constructed URL: https://twitch.tv/{login}
    $stream.thumbnail_url_hd - HD thumbnail (1920x1080)
    """

    def replace_in_value(value):
        if isinstance(value, str):
            # Handle special constructed variables FIRST (before general replacement)
            if stream_data:
                # Thumbnail URL with proper dimensions - handle this before $stream.thumbnail_url
                thumbnail = stream_data.get("thumbnail_url", "").replace("{width}", "1920").replace("{height}", "1080")
                value = value.replace("$stream.thumbnail_url_hd", thumbnail)

            if user_data:
                # Stream URL: https://twitch.tv/username
                value = value.replace("$stream.url", f"https://twitch.tv/{user_data.get('login', '')}")
            elif stream_data:
                value = value.replace("$stream.url", f"https://twitch.tv/{stream_data.get('user_login', '')}")
            else:
                value = value.replace("$stream.url", "https://twitch.tv/")

            # Now handle general variable replacement
            # Replace $user.var_name variables
            if user_data:
                for key, val in user_data.items():
                    variable = f"$user.{key}"
                    if variable in value:
                        if isinstance(val, str):
                            value = value.replace(variable, val)
                        else:
                            value = value.replace(variable, str(val))

            # Replace $stream.var_name variables
            if stream_data:
                for key, val in stream_data.items():
                    variable = f"$stream.{key}"
                    if variable in value:
                        if isinstance(val, str):
                            value = value.replace(variable, val)
                        else:
                            value = value.replace(variable, str(val))

            return value
        elif isinstance(value, dict):
            return {k: replace_in_value(v) for k, v in value.items()}
        elif isinstance(value, list):
            return [replace_in_value(item) for item in value]
        else:
            return value

    return cast(DiscordNotificationMessage, replace_in_value(message_data))


my_message: DiscordNotificationMessage = {
    "content": "Hello $user.display_name, your stream $stream.title is now live!",
    "embed": {
        "title": "$stream.title",
        "description": "$stream.started_at",
        "url": "$stream.url",
        "image": {
            "url": "$stream.thumbnail_url_hd"
        }
    }
}

replace_variables_in_notification(my_message, stream_data, user_data)

{'content': 'Hello escaperoom1_gymnham, your stream Escaperoom Test is now live!',
 'embed': {'title': 'Escaperoom Test',
  'description': '2025-08-17T22:38:49Z',
  'url': 'https://twitch.tv/escaperoom1_gymnham',
  'image': {'url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_escaperoom1_gymnham-1920x1080.jpg'}}}